In [1]:
import pandas as pd
#from IPython.display import display
from sklearn.pipeline import FeatureUnion
#from nltk.corpus import stopwords
#from sklearn.pipeline import Pipeline
#from sklearn.linear_model import LogisticRegression
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.linear_model import LogisticRegression

from capstone_project import preprocessor

#%matplotlib inline

In [2]:
#the current working directory for python is the capstone_project/notebook folder
train_data = pd.read_pickle("../output/data/train_data.pkl")
data.fillna("", inplace=True)

In [3]:
data = data[:30]
tfidf = preprocessor.TfidfTransformer()
feature_egineering = preprocessor.FeatureTransformer()
#clf = LogisticRegression()

y = data["is_duplicate"].values

feature_creator = FeatureUnion([('feature_engineering', feature_egineering), ('tfidf', tfidf)])

feature_creator.fit(data, y)

FeatureUnion(n_jobs=1,
       transformer_list=[('feature_engineering', FeatureTransformer()), ('tfidf', TfidfTransformer())],
       transformer_weights=None)

In [9]:
features = feature_creator.transform(data)
print features.toarray()[0],

[ 33.          31.           0.33333333   0.           0.           0.           0.
   0.           0.           0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.           0.           0.
   0.59497875   0.           0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.           0.           0.
   0.           0.           0.           0.59497875   0.           0.           0.
   0.           0.           0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.           0.      